In [61]:
!which python

/Users/apabook/Desktop/ds-protocol/static/tutorial/venv/bin/python


In [62]:
import json
import requests

# Complete Dataspace Flow


## Consts
Configuration of constant variables used throughout the notebook, including agent URLs and participant IDs.


In [63]:
data_space_provider = "http://127.0.0.1:1200"
data_space_consumer = "http://127.0.0.1:1100"

data_space_provider_participant_id = ""
data_space_provider_participant_token = ""
data_space_provider_dsp_endpoint = ""
data_space_consumer_participant_id = ""
data_space_consumer_participant_token = ""
catalog_id = ""
dataset_id = ""
distribution_id = ""
data_service_id = ""
policy_id = ""
target_id = ""

cn_consumer_id = ""
cn_provider_id = ""
agreement_id = ""
transfer_consumer_id = ""
transfer_provider_id = ""
consumer_data_address = ""

json_header = {
    "Content-Type": "application/json",
}

## Dataspace setup
Authority is created and Consumer Agent and Provider Agent are registered in the Authority. Then, both participants onboard themselves in the system.


### Current Participants Information


In [64]:
url = data_space_provider + "/api/v1/mates/myself"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
data_space_provider_participant_id = response_as_json["participant_id"]
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "did:jwk:provider",
    "participant_slug": "Myself",
    "participant_type": "Agent",
    "base_url": "http://127.0.0.1:1200",
    "token": null,
    "saved_at": "2026-02-08T18:52:01.221800",
    "last_interaction": "2026-02-08T18:52:01.221834",
    "is_me": true
}


In [65]:
url = data_space_consumer + "/api/v1/mates/myself"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
data_space_consumer_participant_id = response_as_json["participant_id"]
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "did:jwk:consumer",
    "participant_slug": "Myself",
    "participant_type": "Agent",
    "base_url": "http://127.0.0.1:1100",
    "token": null,
    "saved_at": "2026-02-08T18:52:03.933141",
    "last_interaction": "2026-02-08T18:52:03.933174",
    "is_me": true
}


### Provider and Consumer IDs
Credential exchange and token retrieval for API calls.


In [66]:
url = data_space_provider + "/api/v1/mates/all"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()

for participant in response_as_json:
    if participant.get("participant_type") == "Agent" and participant.get(
            "participant_id") == data_space_consumer_participant_id:
        data_space_consumer_participant_id = participant["participant_id"]
        data_space_consumer_participant_token = participant["token"]
        break

In [67]:
url = data_space_consumer + "/api/v1/mates/all"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()

for participant in response_as_json:
    if participant.get("participant_type") == "Agent" and participant.get(
            "participant_id") == data_space_provider_participant_id:
        data_space_provider_participant_id = participant["participant_id"]
        data_space_provider_participant_token = participant["token"]
        break

In [68]:
print("Provider DID: {}\n".format(data_space_provider_participant_id))
print("Provider token: {}\n".format(data_space_provider_participant_token))
print("Consumer DID: {}\n".format(data_space_consumer_participant_id))
print("Consumer token: {}".format(data_space_consumer_participant_token))

Provider DID: did:jwk:provider

Provider token: HzX-4U3Cw05--NXRJIoR1gxi3OTgfQ--EXQ2TUbF32c

Consumer DID: did:jwk:consumer

Consumer token: HzX-4U3Cw05--NXRJIoR1gxi3OTgfQ--EXQ2TUbF32c


## Create Dataset, Distributions, Policies and Connector Instances
Provider is going to create a catalog, a data service, a dataset and a distribution to be offered to the consumer. The consumer will then discover the offer and accept it.


In [69]:
url = data_space_provider + "/api/v1/catalog-agent/catalogs/main"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
catalog_id = response_as_json["id"]
print(json.dumps(response_as_json, indent=4))

{
    "dctConformsTo": null,
    "dctCreator": null,
    "dctIdentifier": "urn:catalog:b67a04dd-2dfc-49b8-85aa-48f29deaca72",
    "dctIssued": "2026-02-08T18:52:23.541617Z",
    "dctModified": null,
    "dctTitle": null,
    "dspaceMainCatalog": true,
    "dspaceParticipantId": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiIwLW5uempxdDNQVXZtMUVPaEVxNUNxQ05WcEVUM3I4S0Y3VWRKSDJlRFV1dDNhSFlFWGFxbG51SXZ4TUVmN0J1dkhMVnlUaW95U1BoMV9TaDF0ZnBwdF9aNXpXU1ZZUGp0ekZ0QlJsQ0pQYzNfc2YtelRIQURMMmNyYW84WmxXa1FFR2ZtX1g3Y2QtMkd5bmVuR0k0a01rLU9vRERUWEJNRmVoOVdtR1dyMGZBeHJ4Qkp3Unh4QTExODNIOWo1QkY3dlpVZ0Y3akYzV28xcVlva0Z4emJ0Q2FhWWkzTmE3aFYtWWRIYkg3QUhkYnlJa1JvTHhhVG9tb3ktRHRmb0dSSGRzTmpzNDFqZUwxTG5XczJENnVJMm43QTBFQ1hIaHJjZTlMbkt6eXNMWktRM3Ffa2hxR0UzRTZJT1djajRMT1NvNGJ4SXktZzNVT0NpeElSelJ2bFEifQ",
    "foafHomePage": null,
    "id": "urn:catalog:b67a04dd-2dfc-49b8-85aa-48f29deaca72"
}


In [70]:
url = data_space_provider + "/api/v1/catalog-agent/data-services/main"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
data_service_id = response_as_json["id"]
print(json.dumps(response_as_json, indent=4))

{
    "catalogId": "urn:catalog:b67a04dd-2dfc-49b8-85aa-48f29deaca72",
    "dcatEndpointDescription": null,
    "dcatEndpointUrl": "http://127.0.0.1:1200/dsp/current",
    "dctConformsTo": null,
    "dctCreator": null,
    "dctDescription": null,
    "dctIdentifier": "urn:data-service:1850f547-cf0c-4141-a908-1da7052ef78a",
    "dctIssued": "2026-02-08T18:52:23.622720Z",
    "dctModified": null,
    "dctTitle": null,
    "dspaceMainDataService": true,
    "id": "urn:data-service:1850f547-cf0c-4141-a908-1da7052ef78a"
}


In [71]:
url = data_space_provider + "/api/v1/catalog-agent/datasets"
payload = {
    "dctTitle": "My dataspace dataset",
    "catalogId": catalog_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
dataset_id = response_as_json["id"]
print(json.dumps(response_as_json, indent=4))

{
    "catalogId": "urn:catalog:b67a04dd-2dfc-49b8-85aa-48f29deaca72",
    "dctConformsTo": null,
    "dctCreator": null,
    "dctDescription": null,
    "dctIdentifier": "urn:dataset:7b7c3a8a-9804-43f9-a344-bda2a6309d6e",
    "dctIssued": "2026-02-10T11:45:20.315569Z",
    "dctModified": null,
    "dctTitle": "My dataspace dataset",
    "id": "urn:dataset:7b7c3a8a-9804-43f9-a344-bda2a6309d6e"
}


In [72]:
url = data_space_provider + "/api/v1/catalog-agent/distributions"
payload = {
    "dctTitle": "Distribution of my dataset",
    "dctDescription": "This is the distribution of my dataset",
    "dcatAccessService": data_service_id,
    "datasetId": dataset_id,
    "dctFormats": "http+asd"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
distribution_id = response_as_json["id"]
print(json.dumps(response_as_json, indent=4))

{
    "datasetId": "urn:dataset:7b7c3a8a-9804-43f9-a344-bda2a6309d6e",
    "dcatAccessService": "urn:data-service:1850f547-cf0c-4141-a908-1da7052ef78a",
    "dctDescription": "This is the distribution of my dataset",
    "dctFormat": "http+asd",
    "dctIssued": "2026-02-10T11:45:21.239247Z",
    "dctModified": null,
    "dctTitle": "Distribution of my dataset",
    "id": "urn:distribution:a29c50c2-fe89-495a-a981-68c9ede7ba6b"
}


## Create policies with template


In [73]:
url = data_space_provider + "/api/v1/catalog-agent/policy-templates"
response = requests.request("GET", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

[
    {
        "author": "",
        "content": {
            "permission": [
                {
                    "action": "use",
                    "constraint": [
                        {
                            "leftOperand": "bla",
                            "operator": "eq",
                            "rightOperand": "$bla"
                        }
                    ]
                }
            ]
        },
        "date": "2026-02-08T18:52:23.654816Z",
        "description": "Mi example super policy",
        "id": "policy-1",
        "parameters": {
            "$bla": {
                "dataType": "string",
                "restrictions": {},
                "ui": {
                    "hidden": false
                }
            }
        },
        "title": "Mi example policy",
        "version": "1.0"
    }
]


In [74]:
url = data_space_provider + "/api/v1/catalog-agent/policy-templates/instantiate-odrl-offer"
payload = {
    "id": "policy-1",
    "version": "1.0",
    "parameters": {
        "$bla": "did:web:hola.es"
    },
    "entityId": dataset_id,
    "entityType": "Dataset"
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "createdAt": "2026-02-10T11:45:22.489863Z",
    "entity": "urn:dataset:7b7c3a8a-9804-43f9-a344-bda2a6309d6e",
    "entityType": "Dataset",
    "id": "urn:odrl-policy:fc465f7b-2cae-4924-9386-9e88833c6156",
    "instantiationParameters": {
        "$bla": "did:web:hola.es"
    },
    "odrlOffer": {
        "permission": [
            {
                "action": "use",
                "constraint": [
                    {
                        "leftOperand": "bla",
                        "operator": "eq",
                        "rightOperand": "did:web:hola.es"
                    }
                ]
            }
        ]
    },
    "sourceTemplateId": "policy-1",
    "sourceTemplateVersion": "1.0"
}


## Or... just create policies with API


In [75]:
url = data_space_provider + "/api/v1/catalog-agent/odrl-policies"
payload = {
    "odrlOffer": {
        "permission": [
            {
                "action": "use",
                "constraint": [
                    {
                        "rightOperand": "user",
                        "leftOperand": "did:web:hola.es",
                        "operator": "eq"
                    }
                ]
            }

        ],
        "obligation": [],
        "prohibition": []
    },
    "entityId": dataset_id,
    "entityType": "Dataset"
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "createdAt": "2026-02-10T11:45:24.834951Z",
    "entity": "urn:dataset:7b7c3a8a-9804-43f9-a344-bda2a6309d6e",
    "entityType": "Dataset",
    "id": "urn:odrl-policy:35cc0a1f-d026-4faa-8780-61712dc08adc",
    "instantiationParameters": null,
    "odrlOffer": {
        "obligation": [],
        "permission": [
            {
                "action": "use",
                "constraint": [
                    {
                        "leftOperand": "did:web:hola.es",
                        "operator": "eq",
                        "rightOperand": "user"
                    }
                ]
            }
        ],
        "prohibition": []
    },
    "sourceTemplateId": null,
    "sourceTemplateVersion": null
}


## Create and configure connector to final system



In [76]:
url = data_space_provider + "/api/v1/connector/templates"
payload = {
    "authentication": {
        "type": "NO_AUTH"
    },
    "interaction": {
        "mode": "PULL",
        "dataAccess": {
            "protocol": "HTTP",
            "urlTemplate": "{{__ACCESS_URL__}}",
            "method": "{{__ACCESS_METHODS__}}",
            "headers": "{{__ACCESS_HEADERS__}}"
        }
    },
    "parameters": [
        {
            "paramType": "STRING",
            "name": "ACCESS_URL",
            "title": "Url",
            "required": True
        },
        {
            "paramType": "VEC<STRING>",
            "name": "ACCESS_METHODS",
            "title": "Methods",
            "required": True
        },
        {
            "paramType": "MAP<STRING,STRING>",
            "name": "ACCESS_HEADERS",
            "title": "Methods",
            "required": True
        }
    ]
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
connector_name = response_as_json["name"]
connector_version = response_as_json["version"]
print(json.dumps(response_as_json, indent=4))

{
    "name": "urn:connector-template:5f3217bb-0591-4b0e-9242-0d439600e65c",
    "author": "admin",
    "description": null,
    "version": "1.0",
    "createdAt": "2026-02-10T11:45:25.985250Z",
    "authentication": {
        "type": "NO_AUTH"
    },
    "interaction": {
        "mode": "PULL",
        "dataAccess": {
            "protocol": "HTTP",
            "urlTemplate": "{{__ACCESS_URL__}}",
            "method": "{{__ACCESS_METHODS__}}",
            "headers": "{{__ACCESS_HEADERS__}}",
            "bodyTemplate": null
        }
    },
    "parameters": [
        {
            "name": "ACCESS_URL",
            "title": "Url",
            "description": null,
            "paramType": "STRING",
            "required": true,
            "defaultValue": null,
            "autoFilled": false,
            "autoFilledType": null
        },
        {
            "name": "ACCESS_METHODS",
            "title": "Methods",
            "description": null,
            "paramType": "VEC<STRIN

In [77]:
url = data_space_provider + "/api/v1/connector/instances"
payload = {
    "templateName": connector_name,
    "templateVersion": connector_version,
    "distributionId": distribution_id,
    "metadata": {
        "description": "asasdasddas",
        "ownerId": "asdas"
    },
    "dryRun": False,
    "parameters": {
        "ACCESS_URL": "https://dummyjson.com/products",
        "ACCESS_METHODS": ["GET", "HEAD"],
        "ACCESS_HEADERS": {
            "Content-type": "application/json"
        }
    }
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "id": "urn:connector-instance:53023bb3-24f3-4a59-af66-d9f24ce1ebef",
    "name": "urn:connector-template:5f3217bb-0591-4b0e-9242-0d439600e65c",
    "author": null,
    "description": null,
    "version": "1.0",
    "createdAt": "2026-02-10T11:45:27.619374Z",
    "authenticationConfig": {
        "type": "NO_AUTH"
    },
    "interaction": {
        "mode": "PULL",
        "dataAccess": {
            "protocol": "HTTP",
            "urlTemplate": "https://dummyjson.com/products",
            "method": [
                "GET",
                "HEAD"
            ],
            "headers": {
                "Content-type": "application/json"
            },
            "bodyTemplate": null
        }
    },
    "distributionId": "urn:distribution:a29c50c2-fe89-495a-a981-68c9ede7ba6b"
}


## Consumer discovers the provider catalog and finds the offer


In [78]:
url = data_space_consumer + "/dsp/current/catalog/rpc/setup-catalog-request"
payload = {
    "associatedAgentPeer": data_space_provider_participant_id,
    "filter": [], 
    "noCache": True
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
policy_id = response_as_json["response"]["dataset"][-1]["hasPolicy"][0]["@id"]
target_id = response_as_json["response"]["dataset"][0]["@id"]
data_space_provider_dsp_endpoint = response_as_json["response"]["service"]["endpointURL"]
response_as_json


{'request': {'associatedAgentPeer': 'did:jwk:provider',
  'filter': [],
  'noCache': True},
 'response': {'@context': ['https://w3id.org/dspace/2025/1/context.jsonld'],
  '@type': 'Catalog',
  '@id': 'urn:catalog:b67a04dd-2dfc-49b8-85aa-48f29deaca72',
  'identifier': 'urn:catalog:b67a04dd-2dfc-49b8-85aa-48f29deaca72',
  'issued': '2026-02-08T18:52:23.541617',
  'catalog': [],
  'dataset': [{'@type': 'Dataset',
    '@id': 'urn:dataset:7b7c3a8a-9804-43f9-a344-bda2a6309d6e',
    'identifier': 'urn:dataset:7b7c3a8a-9804-43f9-a344-bda2a6309d6e',
    'issued': '2026-02-10T11:45:20.315569',
    'title': 'My dataspace dataset',
    'hasPolicy': [{'@id': 'urn:odrl-policy:fc465f7b-2cae-4924-9386-9e88833c6156',
      'permission': [{'action': 'use',
        'constraint': [{'rightOperand': 'did:web:hola.es',
          'leftOperand': 'bla',
          'operator': 'eq'}]}],
      '@type': 'Offer'},
     {'@id': 'urn:odrl-policy:35cc0a1f-d026-4faa-8780-61712dc08adc',
      'permission': [{'action': 'u

In [79]:
print("Selected dataset target: \n{}\n".format(target_id))
print("Selected policy id: \n{}\n".format(policy_id))
print("Provider negotiation URL for DSP 2025: \n{}\n".format(data_space_provider_dsp_endpoint))


Selected dataset target: 
urn:dataset:7b7c3a8a-9804-43f9-a344-bda2a6309d6e

Selected policy id: 
urn:odrl-policy:4b083039-3eb4-4158-84be-cf18be39f672

Provider negotiation URL for DSP 2025: 
http://127.0.0.1:1200/dsp/current



## Consumer discovers own callback address based on DSP version

In [80]:
url = data_space_consumer + "/.well-known/dspace-version/2025-1"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
data_space_consumer_dsp_endpoint = data_space_consumer + response_as_json["path"]
print("Consumer callback address: \n{}\n".format(data_space_consumer_dsp_endpoint))


Consumer callback address: 
http://127.0.0.1:1100/dsp/current



## Contract Negotiation

In [109]:
# Initialization of negotiation request (Consumer -> Provider)
url = data_space_consumer + "/dsp/current/negotiations/rpc/setup-request-init"
payload = {
    "associatedAgentPeer": data_space_provider_participant_id,
    "providerAddress": data_space_provider_dsp_endpoint,
    "callbackAddress": data_space_consumer_dsp_endpoint,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "use"
            }
        ]
    }
}
try:
    response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
    response_as_json = response.json()
    cn_consumer_id = response_as_json["response"]["consumerPid"]
    cn_provider_id = response_as_json["response"]["providerPid"]
    print(json.dumps(response_as_json, indent=2))
except Exception as e:
    print("Error in response, {}.".format(e))

{
  "request": {
    "associatedAgentPeer": "did:jwk:provider",
    "providerAddress": "http://127.0.0.1:1200/dsp/current",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current",
    "offer": {
      "@id": "urn:odrl-policy:4b083039-3eb4-4158-84be-cf18be39f672",
      "permission": [
        {
          "action": "use"
        }
      ],
      "@type": "Offer",
      "target": "urn:dataset:7b7c3a8a-9804-43f9-a344-bda2a6309d6e"
    }
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:4559471f-d11c-4a8e-a63c-88e72590b7da",
    "providerPid": "urn:provider-pid:8bcee7b6-e647-457b-b782-4eacd1a2c429",
    "state": "REQUESTED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:aeef6683-9d01-44af-ac43-70a3f7f139dd",
    "state": "REQUESTED",
    "stateAttribute": null,
    "associatedAgentPeer": "did:jwk:provider",
    "protocol": "DSP",
    "callback

In [110]:
# Provider creates initial offer (Provider -> Consumer)
url = data_space_provider + "/dsp/current/negotiations/rpc/setup-offer"
payload = {
    "consumerPid": cn_consumer_id,  # remove to test offer from provider
    "providerPid": cn_provider_id,  # remove to test offer from provider
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "offer": {
      "@id": "urn:odrl-policy:4b083039-3eb4-4158-84be-cf18be39f672",
      "permission": [
        {
          "action": "supermegause"
        }
      ],
      "@type": "Offer",
      "target": "urn:dataset:7b7c3a8a-9804-43f9-a344-bda2a6309d6e"
    },
    "providerPid": "urn:provider-pid:8bcee7b6-e647-457b-b782-4eacd1a2c429",
    "consumerPid": "urn:consumer-pid:4559471f-d11c-4a8e-a63c-88e72590b7da"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:4559471f-d11c-4a8e-a63c-88e72590b7da",
    "providerPid": "urn:provider-pid:8bcee7b6-e647-457b-b782-4eacd1a2c429",
    "state": "OFFERED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:e7b9642d-41c0-445d-bc26-17dbf7f78fc9",
    "state": "OFFERED",
    "stateAttribute": null,
    "associatedAgentPeer": "did:jwk:consumer",
    "protocol": "DSP",
    "callbackAddress": 

In [111]:
# Consumer sends negotiation request based on offer (Consumer -> Provider)
url = data_space_consumer + "/dsp/current/negotiations/rpc/setup-request"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "offer": {
      "@id": "urn:odrl-policy:4b083039-3eb4-4158-84be-cf18be39f672",
      "permission": [
        {
          "action": "use"
        }
      ],
      "@type": "Offer",
      "target": "urn:dataset:7b7c3a8a-9804-43f9-a344-bda2a6309d6e"
    },
    "providerPid": "urn:provider-pid:8bcee7b6-e647-457b-b782-4eacd1a2c429",
    "consumerPid": "urn:consumer-pid:4559471f-d11c-4a8e-a63c-88e72590b7da"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:4559471f-d11c-4a8e-a63c-88e72590b7da",
    "providerPid": "urn:provider-pid:8bcee7b6-e647-457b-b782-4eacd1a2c429",
    "state": "REQUESTED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:aeef6683-9d01-44af-ac43-70a3f7f139dd",
    "state": "REQUESTED",
    "stateAttribute": null,
    "associatedAgentPeer": "did:jwk:provider",
    "protocol": "DSP",
    "callbackAddress": "http

In [112]:
# Provider updates/confirms the offer (Provider -> Consumer)
url = data_space_provider + "/dsp/current/negotiations/rpc/setup-offer"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "offer": {
      "@id": "urn:odrl-policy:4b083039-3eb4-4158-84be-cf18be39f672",
      "permission": [
        {
          "action": "supermegause"
        }
      ],
      "@type": "Offer",
      "target": "urn:dataset:7b7c3a8a-9804-43f9-a344-bda2a6309d6e"
    },
    "providerPid": "urn:provider-pid:8bcee7b6-e647-457b-b782-4eacd1a2c429",
    "consumerPid": "urn:consumer-pid:4559471f-d11c-4a8e-a63c-88e72590b7da"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:4559471f-d11c-4a8e-a63c-88e72590b7da",
    "providerPid": "urn:provider-pid:8bcee7b6-e647-457b-b782-4eacd1a2c429",
    "state": "OFFERED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:e7b9642d-41c0-445d-bc26-17dbf7f78fc9",
    "state": "OFFERED",
    "stateAttribute": null,
    "associatedAgentPeer": "did:jwk:consumer",
    "protocol": "DSP",
    "callbackAddress": 

In [113]:
# Consumer accepts the offer (Consumer -> Provider)
url = data_space_consumer + "/dsp/current/negotiations/rpc/setup-acceptance"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "providerPid": "urn:provider-pid:8bcee7b6-e647-457b-b782-4eacd1a2c429",
    "consumerPid": "urn:consumer-pid:4559471f-d11c-4a8e-a63c-88e72590b7da"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:4559471f-d11c-4a8e-a63c-88e72590b7da",
    "providerPid": "urn:provider-pid:8bcee7b6-e647-457b-b782-4eacd1a2c429",
    "state": "ACCEPTED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:aeef6683-9d01-44af-ac43-70a3f7f139dd",
    "state": "ACCEPTED",
    "stateAttribute": null,
    "associatedAgentPeer": "did:jwk:provider",
    "protocol": "DSP",
    "callbackAddress": "http://127.0.0.1:1200/dsp/current",
    "role": "Consumer",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-10T12:09:01.194569Z",
    "updatedAt": "2026-02-10T12:09:07.248645Z",
    "identifiers": {
      "consumerPid": "urn:consumer-pid:4

In [114]:
# Provider creates the Agreement (Provider -> Consumer)
url = data_space_provider + "/dsp/current/negotiations/rpc/setup-agreement"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "providerPid": "urn:provider-pid:8bcee7b6-e647-457b-b782-4eacd1a2c429",
    "consumerPid": "urn:consumer-pid:4559471f-d11c-4a8e-a63c-88e72590b7da"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:4559471f-d11c-4a8e-a63c-88e72590b7da",
    "providerPid": "urn:provider-pid:8bcee7b6-e647-457b-b782-4eacd1a2c429",
    "state": "AGREED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:e7b9642d-41c0-445d-bc26-17dbf7f78fc9",
    "state": "AGREED",
    "stateAttribute": null,
    "associatedAgentPeer": "did:jwk:consumer",
    "protocol": "DSP",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current",
    "role": "Provider",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-10T12:09:01.098964Z",
    "updatedAt": "2026-02-10T12:09:07.837288Z",
    "identifiers": {
      "providerPid": "urn:provider-pid:8bcee

In [115]:
# Consumer verifies the agreement (Consumer -> Provider)
url = data_space_consumer + "/dsp/current/negotiations/rpc/setup-verification"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "providerPid": "urn:provider-pid:8bcee7b6-e647-457b-b782-4eacd1a2c429",
    "consumerPid": "urn:consumer-pid:4559471f-d11c-4a8e-a63c-88e72590b7da"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:4559471f-d11c-4a8e-a63c-88e72590b7da",
    "providerPid": "urn:provider-pid:8bcee7b6-e647-457b-b782-4eacd1a2c429",
    "state": "VERIFIED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:aeef6683-9d01-44af-ac43-70a3f7f139dd",
    "state": "VERIFIED",
    "stateAttribute": null,
    "associatedAgentPeer": "did:jwk:provider",
    "protocol": "DSP",
    "callbackAddress": "http://127.0.0.1:1200/dsp/current",
    "role": "Consumer",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-10T12:09:01.194569Z",
    "updatedAt": "2026-02-10T12:09:08.501915Z",
    "identifiers": {
      "providerPid": "urn:provider-pid:8

In [116]:
# Provider finalizes the negotiation (Provider -> Consumer)
url = data_space_provider + "/dsp/current/negotiations/rpc/setup-finalization"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    agreement = response_as_json
    agreement_id = response_as_json["negotiationAgentModel"]["agreement"]["id"]
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "providerPid": "urn:provider-pid:8bcee7b6-e647-457b-b782-4eacd1a2c429",
    "consumerPid": "urn:consumer-pid:4559471f-d11c-4a8e-a63c-88e72590b7da"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:4559471f-d11c-4a8e-a63c-88e72590b7da",
    "providerPid": "urn:provider-pid:8bcee7b6-e647-457b-b782-4eacd1a2c429",
    "state": "FINALIZED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:e7b9642d-41c0-445d-bc26-17dbf7f78fc9",
    "state": "FINALIZED",
    "stateAttribute": null,
    "associatedAgentPeer": "did:jwk:consumer",
    "protocol": "DSP",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current",
    "role": "Provider",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-10T12:09:01.098964Z",
    "updatedAt": "2026-02-10T12:09:09.169245Z",
    "identifiers": {
      "providerPid": "urn:provider-pid

In [117]:
print("Final agreement: \n{}\n".format(json.dumps(agreement["negotiationAgentModel"]["agreement"], indent=2)))
print("Final agreement id: \n{}\n".format(agreement_id))

Final agreement: 
{
  "id": "urn:agreement:3983b0f2-6e94-47e5-b67a-59091ce4dfb4",
  "negotiationAgentProcessId": "urn:negotiation-process:e7b9642d-41c0-445d-bc26-17dbf7f78fc9",
  "negotiationAgentMessageId": "urn:negotiation-message:660595b6-6021-4a6d-b0d8-8abb6de9fbd8",
  "consumerParticipantId": "did:jwk:consumer",
  "providerParticipantId": "did:jwk:provider",
  "agreementContent": {
    "@id": "urn:agreement:3983b0f2-6e94-47e5-b67a-59091ce4dfb4",
    "@type": "Agreement",
    "assignee": "did:jwk:consumer",
    "assigner": "did:jwk:provider",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "target": "urn:dataset:7b7c3a8a-9804-43f9-a344-bda2a6309d6e",
    "timestamp": "1770725347"
  },
  "target": "urn:dataset:7b7c3a8a-9804-43f9-a344-bda2a6309d6e",
  "state": "ACTIVE",
  "createdAt": "2026-02-10T12:09:07.850655Z",
  "updatedAt": "2026-02-10T12:09:09.186298Z"
}

Final agreement id: 
urn:agreement:3983b0f2-6e94-47e5-b67a-59091ce4dfb4



## Transfer Negotiation with Agents


In [30]:
# 1. Transfer request setup (Consumer -> Provider)
url = data_space_consumer + "/dsp/current/transfers/rpc/setup-request"
payload = {
    "associatedAgentPeer": data_space_provider_participant_id,
    "providerAddress": data_space_provider_dsp_endpoint,
    "callbackAddress": data_space_consumer_dsp_endpoint,
    "agreementId": agreement_id,
    "format": "http+asd",
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
transfer_process_consumer_pid = response_as_json["response"]["consumerPid"]
transfer_process_provider_pid = response_as_json["response"]["providerPid"]
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "associatedAgentPeer": "did:jwk:provider",
    "agreementId": "urn:agreement:4a064f03-1810-473e-875c-37bf85ca8efd",
    "format": "http+asd",
    "dataAddress": null,
    "providerAddress": "http://127.0.0.1:1200/dsp/current",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:9329f30c-c654-47f8-a904-1941f7e1d2ee",
    "providerPid": "urn:provider-pid:cb7e263b-26b6-4f51-b07e-28e19a008171",
    "state": "REQUESTED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:43139783-c11a-4409-836d-4c1104dc07be",
    "state": "REQUESTED",
    "stateAttribute": "OnRequest",
    "associatedAgentPeer": "",
    "protocol": "DSP",
    "transferDirection": "http+asd",
    "agreementId": "urn:agreement:4a064f03-1810-473e-875c-37bf85ca8efd",
    "callbackAddress": "http://127.0.0.1:1200/dsp/current

In [31]:
# 2. Start transfer (Provider -> Consumer)
url = data_space_provider + "/dsp/current/transfers/rpc/setup-start"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "dataAddress": {
        "@type": "DataAddress",
        "endpointType": "https://w3id.org/idsa/v4.1/HTTP",
        "endpoint": "http://example.com",
        "endpointProperties": [
            {
                "@type": "EndpointProperty",
                "name": "authorization",
                "value": "TOKEN-ABCDEFG"
            },
            {
                "@type": "EndpointProperty",
                "name": "authType",
                "value": "bearer"
            }
        ]
    },
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()

print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:9329f30c-c654-47f8-a904-1941f7e1d2ee",
    "providerPid": "urn:provider-pid:cb7e263b-26b6-4f51-b07e-28e19a008171",
    "dataAddress": {
      "endpointType": "https://w3id.org/idsa/v4.1/HTTP",
      "endpoint": "http://example.com",
      "endpointProperties": [
        {
          "name": "authorization",
          "value": "TOKEN-ABCDEFG"
        },
        {
          "name": "authType",
          "value": "bearer"
        }
      ]
    }
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:9329f30c-c654-47f8-a904-1941f7e1d2ee",
    "providerPid": "urn:provider-pid:cb7e263b-26b6-4f51-b07e-28e19a008171",
    "state": "STARTED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:5df2a8ae-7a97-432c-ae7f-8af754f60138",
    "state": "STARTED",
    "stateAttribute": "OnRequest",
    "associatedAgentPeer": "",
 

In [32]:
# 3. Suspend transfer (Consumer -> Provider)
url = data_space_consumer + "/dsp/current/transfers/rpc/setup-suspension"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "code": "SUSPEND",
    "reason": ["Suspending for demonstration purposes"]
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:9329f30c-c654-47f8-a904-1941f7e1d2ee",
    "providerPid": "urn:provider-pid:cb7e263b-26b6-4f51-b07e-28e19a008171",
    "code": "SUSPEND",
    "reason": [
      "Suspending for demonstration purposes"
    ]
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:9329f30c-c654-47f8-a904-1941f7e1d2ee",
    "providerPid": "urn:provider-pid:cb7e263b-26b6-4f51-b07e-28e19a008171",
    "state": "SUSPENDED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:43139783-c11a-4409-836d-4c1104dc07be",
    "state": "SUSPENDED",
    "stateAttribute": "ByConsumer",
    "associatedAgentPeer": "",
    "protocol": "DSP",
    "transferDirection": "http+asd",
    "agreementId": "urn:agreement:4a064f03-1810-473e-875c-37bf85ca8efd",
    "callbackAddress": "http://127.0.0.1:1200/dsp/current",
    "role": "Consumer",
    "properties": {

In [33]:
# 4. Restart transfer (Consumer -> Provider)
url = data_space_consumer + "/dsp/current/transfers/rpc/setup-start"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()

print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:9329f30c-c654-47f8-a904-1941f7e1d2ee",
    "providerPid": "urn:provider-pid:cb7e263b-26b6-4f51-b07e-28e19a008171",
    "dataAddress": null
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:9329f30c-c654-47f8-a904-1941f7e1d2ee",
    "providerPid": "urn:provider-pid:cb7e263b-26b6-4f51-b07e-28e19a008171",
    "state": "STARTED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:43139783-c11a-4409-836d-4c1104dc07be",
    "state": "STARTED",
    "stateAttribute": "ByConsumer",
    "associatedAgentPeer": "",
    "protocol": "DSP",
    "transferDirection": "http+asd",
    "agreementId": "urn:agreement:4a064f03-1810-473e-875c-37bf85ca8efd",
    "callbackAddress": "http://127.0.0.1:1200/dsp/current",
    "role": "Consumer",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-10T10:23:15.9848

In [34]:
# 5. Suspension by Provider (Provider -> Consumer)
# (Note: Testing provider-initiated suspension)
url = data_space_provider + "/dsp/current/transfers/rpc/setup-suspension"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "code": "SUSPEND",
    "reason": ["Suspending for demonstration purposes"]
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:9329f30c-c654-47f8-a904-1941f7e1d2ee",
    "providerPid": "urn:provider-pid:cb7e263b-26b6-4f51-b07e-28e19a008171",
    "code": "SUSPEND",
    "reason": [
      "Suspending for demonstration purposes"
    ]
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:9329f30c-c654-47f8-a904-1941f7e1d2ee",
    "providerPid": "urn:provider-pid:cb7e263b-26b6-4f51-b07e-28e19a008171",
    "state": "SUSPENDED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:5df2a8ae-7a97-432c-ae7f-8af754f60138",
    "state": "SUSPENDED",
    "stateAttribute": "ByProvider",
    "associatedAgentPeer": "",
    "protocol": "DSP",
    "transferDirection": "http+asd",
    "agreementId": "urn:agreement:4a064f03-1810-473e-875c-37bf85ca8efd",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current",
    "role": "Provider",
    "properties": {

In [35]:
# Failure Test: Attempt start with invalid parameters
url = data_space_consumer + "/dsp/current/transfers/rpc/setup-start"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "dataAddress": None
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:9329f30c-c654-47f8-a904-1941f7e1d2ee",
    "providerPid": "urn:provider-pid:cb7e263b-26b6-4f51-b07e-28e19a008171",
    "dataAddress": null
  },
  "error": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferError",
    "consumerPid": null,
    "providerPid": null,
    "code": "5000",
    "reason": [
      "HTTP Error 400 Bad Request: {\"@context\":[\"https://w3id.org/dspace/2025/1/context.jsonld\"],\"@type\":\"TransferError\",\"consumerPid\":null,\"providerPid\":null,\"code\":\"6030\",\"reason\":[\"TransferProcessMessageType TransferStartMessage is not allowed here. Current state is SUSPENDED ByProvider\",\"Failed to parse file\"]}"
    ]
  }
}


In [36]:
# Failure Test: Attempt duplicate or invalid suspension
url = data_space_consumer + "/dsp/current/transfers/rpc/setup-suspension"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "code": "SUSPEND",
    "reason": ["Suspending for demonstration purposes"]
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:9329f30c-c654-47f8-a904-1941f7e1d2ee",
    "providerPid": "urn:provider-pid:cb7e263b-26b6-4f51-b07e-28e19a008171",
    "code": "SUSPEND",
    "reason": [
      "Suspending for demonstration purposes"
    ]
  },
  "error": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferError",
    "consumerPid": null,
    "providerPid": null,
    "code": "6030",
    "reason": [
      "TransferProcessMessageType TransferSuspensionMessage is not allowed here. Current state is SUSPENDED",
      "Failed to parse file"
    ]
  }
}


In [37]:
# 6. Finalize transfer (Provider -> Consumer)
url = data_space_provider + "/dsp/current/transfers/rpc/setup-completion"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:9329f30c-c654-47f8-a904-1941f7e1d2ee",
    "providerPid": "urn:provider-pid:cb7e263b-26b6-4f51-b07e-28e19a008171"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:9329f30c-c654-47f8-a904-1941f7e1d2ee",
    "providerPid": "urn:provider-pid:cb7e263b-26b6-4f51-b07e-28e19a008171",
    "state": "COMPLETED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:5df2a8ae-7a97-432c-ae7f-8af754f60138",
    "state": "COMPLETED",
    "stateAttribute": "ByProvider",
    "associatedAgentPeer": "",
    "protocol": "DSP",
    "transferDirection": "http+asd",
    "agreementId": "urn:agreement:4a064f03-1810-473e-875c-37bf85ca8efd",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current",
    "role": "Provider",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-10T10:23:15.962649Z",
    "updatedAt"